In [1]:
import hardy.recognition.cnn as cnn
import hardy.recognition.tuner as tuner
import hardy.handling.pre_processing as preprocessing
from hardy.handling import handling as handling
import hardy.handling.to_catalogue as to_catalogue

ModuleNotFoundError: No module named 'transformations'

In [2]:
import pickle

In [3]:
import random

In [9]:
image_data = to_catalogue.save_load_data('rgb_Rex_Imy', file_extension='.npy', load=True)

In [5]:
log_dir = './log/'

In [6]:
config_path = './hardy/recognition/'

In [7]:
def classifier_wrapper(image_data):
    test_set_filenames = preprocessing.hold_out_test_set(image_list=image_data, iterator_mode='arrays')
    test_set_list, learning_set_list = to_catalogue.data_set_split(image_data, test_set_filenames)
    training_set, validation_set = to_catalogue.learning_set(image_list=learning_set_list, iterator_mode='arrays')
    test_set = to_catalogue.test_set(image_list=test_set_list, iterator_mode='arrays')
    model, history = cnn.build_model(training_set, validation_set, config_path=config_path)
    results = cnn.evaluate_model(model, test_set)
    #conf_matrix, report = cnn.report_on_metrics(model, test_set) 
    tuner.report_generation(model, history, results, log_dir, tuner=None, save_model=True, config_path=config_path)

In [8]:
classifier_wrapper(image_data)

Epoch 1/10
182/220 [=======================>......] - ETA: 2:38 - loss: 17.1465 - accuracy: 0.500 - ETA: 1:28 - loss: 6177.4795 - accuracy: 0.484 - ETA: 1:05 - loss: 4519.4455 - accuracy: 0.541 - ETA: 53s - loss: 3427.4944 - accuracy: 0.515 - ETA: 46s - loss: 2743.1425 - accuracy: 0.55 - ETA: 42s - loss: 2350.6356 - accuracy: 0.55 - ETA: 38s - loss: 2017.3762 - accuracy: 0.59 - ETA: 35s - loss: 1767.1785 - accuracy: 0.58 - ETA: 33s - loss: 1574.0540 - accuracy: 0.57 - ETA: 31s - loss: 1417.5946 - accuracy: 0.58 - ETA: 30s - loss: 1289.1214 - accuracy: 0.59 - ETA: 29s - loss: 1181.7161 - accuracy: 0.62 - ETA: 28s - loss: 1091.0186 - accuracy: 0.62 - ETA: 27s - loss: 1013.3606 - accuracy: 0.61 - ETA: 26s - loss: 946.1458 - accuracy: 0.5917 - ETA: 26s - loss: 887.1112 - accuracy: 0.599 - ETA: 25s - loss: 835.0322 - accuracy: 0.599 - ETA: 24s - loss: 788.7375 - accuracy: 0.590 - ETA: 24s - loss: 747.2766 - accuracy: 0.582 - ETA: 23s - loss: 709.9371 - accuracy: 0.589 - ETA: 23s - loss: 676

190/220 [========================>.....] - ETA: 18s - loss: 0.5407 - accuracy: 0.656 - ETA: 18s - loss: 0.5258 - accuracy: 0.671 - ETA: 18s - loss: 0.5308 - accuracy: 0.666 - ETA: 18s - loss: 0.5553 - accuracy: 0.632 - ETA: 18s - loss: 0.5292 - accuracy: 0.681 - ETA: 20s - loss: 0.5511 - accuracy: 0.666 - ETA: 21s - loss: 0.5499 - accuracy: 0.674 - ETA: 21s - loss: 0.5509 - accuracy: 0.679 - ETA: 21s - loss: 0.5449 - accuracy: 0.687 - ETA: 20s - loss: 0.5562 - accuracy: 0.668 - ETA: 20s - loss: 0.5400 - accuracy: 0.684 - ETA: 19s - loss: 0.5364 - accuracy: 0.687 - ETA: 19s - loss: 0.5345 - accuracy: 0.689 - ETA: 19s - loss: 0.5341 - accuracy: 0.696 - ETA: 20s - loss: 0.5414 - accuracy: 0.685 - ETA: 20s - loss: 0.5488 - accuracy: 0.675 - ETA: 20s - loss: 0.5510 - accuracy: 0.674 - ETA: 20s - loss: 0.5522 - accuracy: 0.675 - ETA: 20s - loss: 0.5547 - accuracy: 0.672 - ETA: 20s - loss: 0.5578 - accuracy: 0.670 - ETA: 20s - loss: 0.5584 - accuracy: 0.669 - ETA: 20s - loss: 0.5555 - accurac

220/220 [==============================] - ETA: 2s - loss: 0.5503 - accuracy: 0.67 - ETA: 2s - loss: 0.5504 - accuracy: 0.67 - ETA: 2s - loss: 0.5499 - accuracy: 0.68 - ETA: 2s - loss: 0.5495 - accuracy: 0.68 - ETA: 2s - loss: 0.5498 - accuracy: 0.68 - ETA: 2s - loss: 0.5499 - accuracy: 0.68 - ETA: 2s - loss: 0.5501 - accuracy: 0.67 - ETA: 2s - loss: 0.5500 - accuracy: 0.68 - ETA: 1s - loss: 0.5502 - accuracy: 0.67 - ETA: 1s - loss: 0.5498 - accuracy: 0.67 - ETA: 1s - loss: 0.5496 - accuracy: 0.67 - ETA: 1s - loss: 0.5497 - accuracy: 0.67 - ETA: 1s - loss: 0.5497 - accuracy: 0.67 - ETA: 1s - loss: 0.5494 - accuracy: 0.68 - ETA: 1s - loss: 0.5499 - accuracy: 0.67 - ETA: 1s - loss: 0.5498 - accuracy: 0.67 - ETA: 1s - loss: 0.5497 - accuracy: 0.68 - ETA: 1s - loss: 0.5493 - accuracy: 0.68 - ETA: 1s - loss: 0.5496 - accuracy: 0.67 - ETA: 0s - loss: 0.5500 - accuracy: 0.67 - ETA: 0s - loss: 0.5494 - accuracy: 0.67 - ETA: 0s - loss: 0.5495 - accuracy: 0.67 - ETA: 0s - loss: 0.5498 - accuracy

191/220 [=========================>....] - ETA: 18s - loss: 0.6027 - accuracy: 0.593 - ETA: 18s - loss: 0.5569 - accuracy: 0.656 - ETA: 18s - loss: 0.5768 - accuracy: 0.645 - ETA: 17s - loss: 0.5605 - accuracy: 0.664 - ETA: 17s - loss: 0.5416 - accuracy: 0.675 - ETA: 18s - loss: 0.5390 - accuracy: 0.677 - ETA: 18s - loss: 0.5223 - accuracy: 0.696 - ETA: 18s - loss: 0.5209 - accuracy: 0.699 - ETA: 18s - loss: 0.5216 - accuracy: 0.701 - ETA: 18s - loss: 0.5219 - accuracy: 0.690 - ETA: 18s - loss: 0.5263 - accuracy: 0.684 - ETA: 18s - loss: 0.5252 - accuracy: 0.692 - ETA: 18s - loss: 0.5288 - accuracy: 0.692 - ETA: 18s - loss: 0.5328 - accuracy: 0.687 - ETA: 18s - loss: 0.5314 - accuracy: 0.689 - ETA: 18s - loss: 0.5329 - accuracy: 0.685 - ETA: 18s - loss: 0.5350 - accuracy: 0.682 - ETA: 18s - loss: 0.5346 - accuracy: 0.684 - ETA: 18s - loss: 0.5381 - accuracy: 0.680 - ETA: 18s - loss: 0.5398 - accuracy: 0.681 - ETA: 17s - loss: 0.5441 - accuracy: 0.678 - ETA: 17s - loss: 0.5454 - accurac

220/220 [==============================] - ETA: 2s - loss: 0.5498 - accuracy: 0.67 - ETA: 2s - loss: 0.5502 - accuracy: 0.67 - ETA: 2s - loss: 0.5497 - accuracy: 0.67 - ETA: 2s - loss: 0.5493 - accuracy: 0.67 - ETA: 2s - loss: 0.5488 - accuracy: 0.68 - ETA: 1s - loss: 0.5489 - accuracy: 0.68 - ETA: 1s - loss: 0.5491 - accuracy: 0.68 - ETA: 1s - loss: 0.5490 - accuracy: 0.67 - ETA: 1s - loss: 0.5496 - accuracy: 0.67 - ETA: 1s - loss: 0.5499 - accuracy: 0.67 - ETA: 1s - loss: 0.5502 - accuracy: 0.67 - ETA: 1s - loss: 0.5500 - accuracy: 0.67 - ETA: 1s - loss: 0.5498 - accuracy: 0.67 - ETA: 1s - loss: 0.5499 - accuracy: 0.67 - ETA: 1s - loss: 0.5499 - accuracy: 0.67 - ETA: 1s - loss: 0.5497 - accuracy: 0.67 - ETA: 1s - loss: 0.5498 - accuracy: 0.67 - ETA: 0s - loss: 0.5501 - accuracy: 0.67 - ETA: 0s - loss: 0.5501 - accuracy: 0.67 - ETA: 0s - loss: 0.5500 - accuracy: 0.67 - ETA: 0s - loss: 0.5500 - accuracy: 0.67 - ETA: 0s - loss: 0.5498 - accuracy: 0.68 - ETA: 0s - loss: 0.5499 - accuracy

In [8]:
def hold_out_test_set(path=None, number_of_files_per_class=100, classes=['noise', ''],
                      file_extension='.csv', image_list=None, iterator_mode=None):
    '''
    Functions that returns a list of filenames
    of the randomly selected files to compose the test set

    Parameters
    ----------
    path : str
           string containing the path to the files to select from
           the test set from.

    number_of_files_per_class: int
                               The number of files to select from each class.

    classes: list
             a list containing strings of the classes the data is divided in.
             The classes are contained in the filename as labels.

    file_extension: str
                    the extension of the file to read. The default value is
                    .csv
    image_list: np.array
                numpy array representing file names, image data and labels
    iterator_mode: str
                   string representing if the data provided is in arrays

    Returns
    -------
    test_set_serialnumbers : list
                             A list containig the strings of filenames
                             randomly selected to be part of the test set.
    '''

    # Initialize a list that will contain the serial numbers of thefiles
    # composing the test set
    test_set_filenames = []
    
    # seperating test_set_filenames for input as arrays
    if iterator_mode == "arrays":
        file_list_1 = [n[0] for n in image_data if n[0].endswith(classes[0])]
        file_list_2 = [n[0] for n in image_data if not n[0].endswith(classes[0])]
        for i in range(number_of_files_per_class):
            chosen_file = random.choice(file_list_1)
            file_list_1.remove(chosen_file)
            test_set_filenames.append(str(chosen_file))
            chosen_file = random.choice(file_list_2)
            file_list_2.remove(chosen_file)
            test_set_filenames.append(str(chosen_file))
    # # These lines are hardcoded to allow for 2 classes only # #
    # #  Rewrite to support a higher number of classes # #

    # Randomly pick files that are labelled as noisy and append
    #  them into the test_set list
    else:
        file_list_1 = [n for n in os.listdir(path)
                    if n.endswith(classes[0]+file_extension)]
        file_list_2 = [n for n in os.listdir(path)
                    if not n.endswith(classes[0]+file_extension)]
        for i in range(number_of_files_per_class):
            chosen_file = random.choice(file_list_1)
            file_list_1.remove(chosen_file)
            test_set_filenames.append(str(chosen_file.rstrip(chosen_file[-4:])
                                        ))

            chosen_file = random.choice(file_list_2)
            file_list_2.remove(chosen_file)
            test_set_filenames.append(str(chosen_file.rstrip(chosen_file[-4:])
                                        ))

    return test_set_filenames